# Toxicity Detection Model

Get live data with `get_comments.py` and transform Kaggle data set with `transform_train.py`

Get libraries by running `pip install -r requirements.txt`

This is a deep learning model.

In [27]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import STOPWORDS, wordcloud
import re
from fuzzywuzzy import fuzz, process
from nltk.corpus import stopwords

from tqdm import tqdm

import math

# Load Dataset

In [2]:
df_train = pd.read_csv('./data/new_train.csv')

### Data Cleansing
<ul>
    <li>Removal of special characters</li>
    <li>Expanding contractions</li>
    <li>Lowering text</li>
    <li>Replacing Obfuscated Profane Words</li>
</ul>

In [4]:
def decontracted(phrase):
    '''
    This function decontracts words like won't to will not
    '''

    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

In [5]:
def removeNonPrintable(com):
    com = com.replace('\\r', ' ')
    com = com.replace('\\n', ' ')
    com = com.replace('\\t', ' ')
    com = com.replace('\\"', ' ')
    return com

In [6]:
def getUniqueWords(comments):
    unique_words = set()
    for comment in tqdm(comments):
        words = comment.split(" ")
        for word in words:
            if len(word) > 2:
                unique_words.add(word)
    
    return unique_words

In [7]:
def getProfaneWords():
    profane_words = []
    with open("./data/bad-words.txt","r") as f:
        for word in f:
            word = word.replace("\n","")
            profane_words.append(word)
    return profane_words
    

In [8]:
def createMappingDict(profane_words, unique_words):
    # mapping dictionary
    mapping_dict = dict()
    
    # looping through each profane word
    for profane in tqdm(profane_words):
        mapped_words = set()
        
        # looping through each word in vocab
        for word in unique_words:
            # mapping only if ratio > 80
            try:
                if fuzz.ratio(profane,word) > 80:
                    mapped_words.add(word)
            except:
                pass
                
        # list of all vocab words for given profane word
        mapping_dict[profane] = mapped_words
    
    return mapping_dict

In [9]:
def replaceWords(corpus, mapping_dict):
    processed_corpus = []

    for document in tqdm(corpus):

        # words = document.split()

        for mapped_word, v in mapping_dict.items():
            
            document = re.sub(r'\b{word}\b'.format(word = v), mapped_word, document)

            # for target_word in v:

            #     for i, word in enumerate(words):
            #         if word == target_word:
            #             words[i] = mapped_word

        # document = " ".join(words)
        document = document.strip()

        processed_corpus.append(document)

    return processed_corpus



In [10]:
def final_processing(corpus):
    '''
    Function applies final processing steps post profane mapping such as removing special characters,
    punctuations etc.
    '''
    processed_comments = []
    stopwords_list = stopwords.words("english")
    print('final_processing')

    emoj = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)


    # looping through each comment in corpus
    for comment in tqdm(corpus):
        try:
          processed_words = []
          comment = re.sub(emoj, r'', comment)
          comment = re.sub(r'http\S+', '', comment)
          # comment = re.sub("\B\#\w+", ' ', comment)
          # comment = re.sub("\B\@\w+", '', comment)
          comment = re.sub(r'(\w*#\w+|\w+#\w*)','', comment)
          comment = re.sub(r'(\w*@\w+|\w+#\w*)','',  comment)
          comment = re.sub(r'[^A-Za-z\s]+',"",  comment) # retain only letters
          for word in comment.split():
              if len(word) >= 3:
                  processed_words.append(word)
          
          processed_comments.append(comment.strip())
        except Exception as e:
          print(corpus)
          print(e)
          pass
    
    return processed_comments

In [11]:
def cleanComments(comments):
    processed_comments = []
    for comment in comments:
        comment = decontracted(comment)
        comment = removeNonPrintable(comment)

        # Lower comment
        processed_comments.append(comment.lower().strip())
      
    profane_words = getProfaneWords()
    #unique_words = getUniqueWords(processed_comments)
    #profane_dict = createMappingDict(profane_words, unique_words)
    #processed_comments = replaceWords(processed_comments, profane_dict)
    final_comments = final_processing(processed_comments)
    return final_comments

In [12]:
def corrHeatmap(df, fileName = None):
    classes = df.columns[1:]
    data = df.copy()
    data = data[classes]
    corr = data.corr()

    plt.figure(figsize=(8,6))
    sns.heatmap(corr,annot=True,vmin=1,vmax=0,fmt='.2g',cmap='rocket')
    plt.title("Correlation Matrix: Labels of Comments")
    if fileName:
        plt.savefig(fileName + '.png')
    plt.show()

In [13]:
def showPercentiles(data):
    print("========== For 0-100 ==========")
    for i in range(11):
        print(f'{i*10}th Percentile Value = {np.percentile(data, i*10)}')

    print("\n")
    print("========== For 90-100 ==========")
    for i in range(11):
        print(f'{90+i}th Percentile Value = {np.percentile(data, 90 + i)}')

In [14]:
def plotHist(data, bin_size, title, fileName = None, columns = None):
    if columns is not None:
        fig, axis = plt.subplots(len(columns)-1, 1, figsize = (12,30))
        for i, col in enumerate(columns[1:]):
            sns.histplot(data=data[col],bins=bin_size,palette="rocket",ax = axis[i])

        fig.tight_layout()
        fig.subplots_adjust(top = 0.95)
        fig.suptitle(title, size = 18)
    else:
        sns.histplot(data=data,bins=bin_size,palette="rocket")
        plt.title(title)
    

    if fileName:
        plt.savefig(fileName + '.png')
    
    plt.show()

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
plotHist(df_train, 30, 'Histogram of the Labels', 'categories_histogram', df_train.columns)

In [ ]:
plotHist(df_train.Comment.str.len(), 50, 'Length of the Comments', 'comments_length_before_preprocess')

In [ ]:
for cat in df_train.columns[1:]:
    print(f"============== {cat} ==============")
    showPercentiles(df_train[cat])
    print("\n")

In [ ]:
showPercentiles(df_train.Comment.str.len())

In [ ]:
corrHeatmap(df_train, 'corr_matrix_labels')

There is a strong positive relationship between Toxicity-Severe Toxicity, Identity Attack-Insult and Toxicity-Profanity respectively

In [15]:
processed_comments = cleanComments(df_train.Comment.values)

final_processing


100%|██████████| 132291/132291 [00:21<00:00, 6041.94it/s]


In [16]:
df_processed = df_train[df_train.columns[1:]].copy()
df_processed['ProcessedComment'] = processed_comments

In [ ]:
df_processed = pd.read_csv('./data/new_train_10k.csv')
df_processed.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
df_processed.head()

In [17]:
cols = df_processed.columns.tolist()
ordered_cols = cols[-1:] + cols[:-1]
df_processed = df_processed[ordered_cols]

In [ ]:
df_processed.head()

In [ ]:
df_processed.shape

In [ ]:
plotHist(df_processed.ProcessedComment.str.len(), 50, 'Comments Length After Preprocessing', 'after_preprocess_comments_histogram')

As seen from the histogram above, the length most of the comments have is in between 0-300. Additionally, the histogram fits to logarithmic distribution and it is right-skewed. To examine the histogram furher, we can look into percentiles of the histogram to determine the vector dimensions which will be crucial for our deep learning model

In [ ]:
showPercentiles(df_processed.ProcessedComment.str.len())

Almost 95% of the comments have 1000 characters approximately. As a result, the input dimension of the tokenized vectors can be 1000.

# Model Building

Turn dataset into list(list of tokens, scores x6))

In [18]:
VOCAB_SIZE = 1000
tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE)

In [20]:
processed_comments = df_processed.dropna().loc[:,"ProcessedComment"].convert_dtypes(infer_objects=True).array
detections = df_processed.dropna().loc[:,"Toxicity": "Threat"].convert_dtypes(infer_objects=True).__array__()

In [21]:
tokenizer.fit_on_texts(processed_comments)
tokenized_comments = tokenizer.texts_to_sequences(processed_comments)

In [22]:
for tokens in tokenized_comments:
	tokens.extend([0] * (VOCAB_SIZE - len(tokens)))

In [23]:
train_input = np.asarray([np.array(input[:VOCAB_SIZE], dtype=int) for input in tokenized_comments[: int(len(tokenized_comments) * .9) ]])
train_output = np.asarray(detections[: int(len(tokenized_comments) * .9) ], dtype=np.float32)

test_input = np.asarray([np.array(input[:VOCAB_SIZE], dtype=int) for input in tokenized_comments[int(len(tokenized_comments) * .9) :]])
test_output = np.asarray(detections[int(len(tokenized_comments) * .9) :], dtype=np.float32)

In [24]:
model = keras.Sequential()

model.add(layers.Dense(64, input_dim=VOCAB_SIZE, kernel_initializer='he_uniform', activation='relu'))
model.add(layers.Dense(32, kernel_initializer='he_uniform'))
model.add(layers.Dense(6))

model.compile(loss = keras.losses.BinaryCrossentropy(from_logits = True),
			  optimizer = keras.optimizers.adam_v2.Adam(1e-4))

In [25]:
history = model.fit(x=train_input,
					y=train_output,
					epochs = 100,
					steps_per_epoch = math.floor(train_input.shape[0]/100),
					verbose=1,
					validation_data=(test_input, test_output))

Epoch 1/100
1190/1190 [==============================] - 4s 3ms/step - loss: 3.4901 - val_loss: 1.4328
Epoch 2/100
1190/1190 [==============================] - 3s 3ms/step - loss: 0.9819 - val_loss: 0.6357
Epoch 3/100
1190/1190 [==============================] - 3s 3ms/step - loss: 0.4603 - val_loss: 0.3390
Epoch 4/100
1190/1190 [==============================] - 3s 3ms/step - loss: 0.2799 - val_loss: 0.2451
Epoch 5/100
1190/1190 [==============================] - 3s 3ms/step - loss: 0.2255 - val_loss: 0.2159
Epoch 6/100
1190/1190 [==============================] - 4s 3ms/step - loss: 0.2080 - val_loss: 0.2063
Epoch 7/100
1190/1190 [==============================] - 3s 2ms/step - loss: 0.2015 - val_loss: 0.2012
Epoch 8/100
1190/1190 [==============================] - 3s 2ms/step - loss: 0.1974 - val_loss: 0.1976
Epoch 9/100
1190/1190 [==============================] - 3s 2ms/step - loss: 0.1954 - val_loss: 0.1960
Epoch 10/100
1190/1190 [==============================] - 3s 2ms/step - l

In [28]:
# model.save("./model")

INFO:tensorflow:Assets written to: ./model\assets


In [29]:
test_loss, test_acc = model.evaluate(x=test_input, y=test_output)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

414/414 [==============================] - 1s 2ms/step - loss: 0.1933


TypeError: cannot unpack non-iterable float object